# Calculate the performonce on every feature

In [1]:
import bz2
import os
import re
import math
import configparser

import pandas
import sklearn.metrics
import scipy.stats
from statsmodels.sandbox.stats.multicomp import multipletests

## Read features and partitions

In [2]:
# Read partition information
part_df = pandas.read_table('data/partitions.tsv')
part_df.tail(2)

,hetnet,compound_id,disease_id,status
22648,rephetio-v2.0,DB08906,DOID:9970,0
22649,rephetio-v2.0_perm-2,DB08906,DOID:9970,0


In [3]:
# Read DWPC results
dwpc_df = pandas.read_table('data/dwpc.tsv.bz2')
dwpc_df = dwpc_df.rename(columns={'metapath': 'feature', 'DWPC': 'value'})
dwpc_df['feature_type'] = 'DWPC'
dwpc_df.tail(2)

,hetnet,compound_id,disease_id,feature,PC,w,value,seconds,feature_type
27308956,rephetio-v2.0,DB08906,DOID:9970,CdGeAeGaD,0,0.4,0.0,0.004798,DWPC
27308957,rephetio-v2.0_perm-2,DB08906,DOID:9970,CdGeAeGaD,0,0.4,0.0,0.252900,DWPC


In [4]:
config = configparser.ConfigParser()
config.read('../config.ini')
commit = config['hetnet']['integrate_commit']

url = 'https://github.com/dhimmel/integrate/raw/{}/data/summary/degrees.xlsx'.format(commit)
disease_degree_df = pandas.read_excel(url, sheetname='Disease')
disease_degree_df = disease_degree_df.rename(columns={'node_id': 'disease_id'}).drop('node_name', axis='columns')
compound_degree_df = pandas.read_excel(url, sheetname='Compound')
compound_degree_df = compound_degree_df.rename(columns={'node_id': 'compound_id'}).drop('node_name', axis='columns')

url = 'https://github.com/dhimmel/integrate/raw/{}/data/summary/metaedge-styles.tsv'.format(commit)
metaedge_style_df = pandas.read_table(url)
metaedge_to_abbreviation = dict(zip(metaedge_style_df.metaedge, metaedge_style_df.abbreviation))

In [5]:
degree_df = part_df.merge(compound_degree_df).merge(disease_degree_df)
degree_vars = list(compound_degree_df.columns[1:]) + list(disease_degree_df.columns[1:])
degree_df = pandas.melt(degree_df, id_vars=['hetnet', 'compound_id', 'disease_id'],
    value_vars=degree_vars, var_name='feature')
degree_df['feature'] = degree_df['feature'].map(metaedge_to_abbreviation)
degree_df['feature_type'] = 'degree'
degree_df.tail(2)

,hetnet,compound_id,disease_id,feature,value,feature_type
362398,rephetio-v2.0_perm-4,DB08882,DOID:8778,DuG,250,degree
362399,rephetio-v2.0_perm-5,DB08906,DOID:8778,DuG,250,degree


In [6]:
feature_df = part_df.merge(pandas.concat([degree_df, dwpc_df]))
feature_df.head(2)

,hetnet,compound_id,disease_id,status,PC,feature,feature_type,seconds,value,w
0,rephetio-v2.0,DB00014,DOID:0050741,0,NaN,CbG,degree,NaN,2.0,NaN
1,rephetio-v2.0,DB00014,DOID:0050741,0,NaN,CcSE,degree,NaN,249.0,NaN


## Compute performance

In [7]:
def compute_metrics(df):
    y_true = df['status']
    y_score = df['value']
    series = pandas.Series()
    series['nonzero'] = (y_score > 0).mean()
    series['auroc'] = sklearn.metrics.roc_auc_score(y_true, y_score)
    series['auprc'] = sklearn.metrics.average_precision_score(y_true, y_score)
    return series

auc_df = feature_df.groupby(['hetnet', 'feature_type', 'feature']).apply(compute_metrics).reset_index()
auc_df['permuted'] = auc_df.hetnet.str.contains('_perm').astype(int)

In [8]:
auc_df.head(2)

,hetnet,feature_type,feature,nonzero,auroc,auprc,permuted
0,rephetio-v2.0,DWPC,CbG<rG<rGaD,0.841854,0.641607,0.329948,0
1,rephetio-v2.0,DWPC,CbG<rG<rGdD,0.379073,0.582892,0.280091,0


In [9]:
def compare_permutation(df):
    unperm = df.query("permuted == 0").iloc[0, :]
    perm_df = df.query("permuted == 1")
    series = pandas.Series()
    series['nonzero'] = unperm['nonzero']
    series['auroc'] = unperm.auroc
    series['auroc_permuted'] = perm_df.auroc.mean()
    series['delta_auroc'] = series['auroc'] - series['auroc_permuted']
    ttest = scipy.stats.ttest_1samp(perm_df.auroc, unperm.auroc)
    pvalue = ttest.pvalue
    series['pval_auroc'] = pvalue
    #series['nlog10_pval_auroc'] = -math.log10(pvalue)
    return(series)

compare_df = auc_df.groupby(['feature_type', 'feature']).apply(compare_permutation).reset_index()
reject, compare_df['fdr_pval_auroc'], alphacSidak, alphacBonf = multipletests(
    pvals=compare_df.pval_auroc, method='fdr_bh')
compare_df = compare_df.sort_values('feature')

In [10]:
compare_df.tail(3)

,feature_type,feature,nonzero,auroc,auroc_permuted,delta_auroc,pval_auroc,fdr_pval_auroc
1219,degree,DrD,0.964238,0.478469,0.467968,0.010501,0.001071,0.007565
1220,degree,DtC,1.000000,0.755264,0.764825,-0.009561,0.008591,0.030343
1221,degree,DuG,0.500397,0.578677,0.581485,-0.002808,0.095563,0.168269


In [11]:
len(compare_df)

1222

In [12]:
compare_df.sort_values('pval_auroc', ascending=True).head(5)

,feature_type,feature,nonzero,auroc,auroc_permuted,delta_auroc,pval_auroc,fdr_pval_auroc
529,DWPC,CiPCiCtD,0.133510,0.755451,0.524214,0.231238,1.746641e-08,0.000021
815,DWPC,CtDaGaD,0.855629,0.610896,0.494765,0.116132,9.921392e-08,0.000045
779,DWPC,CrCtDrDrD,0.278940,0.634317,0.499921,0.134396,1.093963e-07,0.000045
767,DWPC,CrCtD,0.123974,0.749600,0.524350,0.225250,2.356989e-07,0.000072
55,DWPC,CbGbCtD,0.729801,0.866665,0.667216,0.199448,2.998519e-07,0.000073


In [13]:
# Save datasets
auc_df.to_csv('data/auc.tsv', sep='\t', index=False, float_format='%.5g')
compare_df.to_csv('data/auroc.tsv', sep='\t', index=False, float_format='%.5g')

## Create matrix

In [14]:
# Read compound and disease degrees
compound_df = pandas.read_table('../summary/compounds.tsv')
compound_df = compound_df.iloc[:, :2]
disease_df = pandas.read_table('../summary/diseases.tsv')
disease_df = disease_df.iloc[:, :2]

In [15]:
# Create spread dataframes
# compound-disease pairs as rows, metapaths as columns
for hetnet in feature_df.hetnet.unique():
    print(hetnet)
    df = part_df.merge(feature_df.query("hetnet == @hetnet"))
    df = pandas.pivot_table(df, values='value', index=['compound_id', 'disease_id', 'status'], columns='feature')
    df = df.reset_index()
    df = compound_df.merge(disease_df.merge(df))
    for feature in compare_df.query("feature_type == 'degree'").feature:
        df[feature] = df[feature].astype(int)
    directory = 'data/matrix/{}'.format(hetnet)
    if not os.path.exists(directory):
        os.mkdir(directory)
    filename = 'features.tsv.bz2'
    path = os.path.join(directory, filename)
    with bz2.open(path, 'wt') as wf:
        df.to_csv(wf, index=False, sep='\t')

rephetio-v2.0
rephetio-v2.0_perm-5
rephetio-v2.0_perm-1
rephetio-v2.0_perm-3
rephetio-v2.0_perm-2
rephetio-v2.0_perm-4


In [16]:
df.head(2)

,compound_id,compound_name,disease_id,disease_name,status,CbG,CbG<rG<rGaD,CbG<rG<rGdD,CbG<rG<rGuD,CbG<rGaD,...,CuGuDuGdD,CuGuDuGuD,DaG,DdG,DlA,DpC,DpS,DrD,DtC,DuG
0,DB01048,Abacavir,DOID:5408,Paget's disease of bone,0,3,0.0,0.0,0.0,0.0,...,0.0,0.0,16,0,26,0,22,6,6,0
1,DB01048,Abacavir,DOID:2841,asthma,0,3,0.0,0.0,0.0,0.0,...,0.0,0.0,222,0,23,3,26,3,37,4
